In [ ]:
import pandas as pd
import os
import torch
import torch.nn as nn
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import random
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch.nn.functional as F
from datetime import datetime

# Task#1: Variability prediction

### Data preprocessing

In [2]:
# Helper function to clean and fix variability labels
def clean_variability_label(label):
    label = label.lower()
    if "periodic" in label:
        return "periodic"
    elif "aperiodic" in label:
        return "aperiodic"
    elif "not" in label:
        return "not variable"
    else:
        print(label)

class LightCurveDataset(Dataset):
    def __init__(self, files, sequence_length=1000):
        self.files = files
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        # Load CSV file (skip metadata rows)
        df = pd.read_csv(self.files[idx], skiprows=9)

        # Extract date, time, StdMag
        df['Timestamp'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], errors='coerce')
        df['Timestamp'] = df['Timestamp'].ffill()
        #df = df.dropna(subset=['Timestamp']) #filling이 아니라 drop하고 싶다면 대체
        df = df.sort_values('Timestamp')
        df['TimeDiff'] = df['Timestamp'].diff().dt.total_seconds().fillna(0)

        # Convert to tensors
        stdmag = torch.tensor(df['StdMag'].values, dtype=torch.float32)
        time_diff = torch.tensor(df['TimeDiff'].values, dtype=torch.float32)

        # Pad or truncate sequences to fixed length
        seq_len = len(stdmag)
        if seq_len > self.sequence_length:
            start_idx = random.randint(0, seq_len - self.sequence_length)
            stdmag = stdmag[start_idx:start_idx + self.sequence_length]
            time_diff = time_diff[start_idx:start_idx + self.sequence_length]
            seq_len = self.sequence_length
        else:
            padding = self.sequence_length - seq_len
            stdmag = torch.cat([stdmag, torch.zeros(padding)])
            time_diff = torch.cat([time_diff, torch.zeros(padding)])

        # Variability type label
        try:
            metadata = pd.read_csv(self.files[idx], nrows=5, header=None)
            variability_type = metadata.iloc[3, 1]
            variability_type = clean_variability_label(str(variability_type))
        except (IndexError, AttributeError, KeyError):
            variability_type = 'not variable'
        label = torch.tensor(label_encoder.transform([variability_type])[0], dtype=torch.long)

        return time_diff, stdmag, label

NameError: name 'Dataset' is not defined